In [93]:
import django_initializer
from api.models.user.user import User
from django.contrib.auth.models import User as AuthUser
from json import loads, dumps
from collections import OrderedDict
import json
import re
from api.models.campaign.campaign import Campaign
from api.models.user.user_subscription import UserSubscription
def to_dict(input_ordered_dict):
    return loads(dumps(input_ordered_dict))

In [218]:
import pickle
from django.conf import settings
import functools, logging, traceback
import os
import time
import json
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains, Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.color import Color
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import arrow


class FacebookSharedListCrawler():
    def __init__(self, page_username, target_post_id, lang="en", chrome_driver_path=os.path.join(settings.BASE_DIR, 'chromedriver.exe')):
        self.chrome_options = Options() 
#         self.chrome_options.add_argument('--headless')  # 啟動Headless 無頭
        self.chrome_options.add_argument('--disable-gpu')
        
        self.chromeService = Service(executable_path=chrome_driver_path)

        self.driver = None #套用設定
        self.wait = None
        self.actions = None
        
        self.login_url = "https://m.facebook.com/"
        self.page_url = f'https://m.facebook.com/{page_username}/'
        self.target_post_id = target_post_id
        
        self.post_reference_id = ""
        self.post_shared_list_url = "https://m.facebook.com/browse/shares?id="
        
        self.email = "twadmin@algotech.app"
        self.password = "algoFB2021"
        self.loop_count = 0
        self.save_login = False
        self.is_login = False
        self.cookies = None
        self.cookies_path = os.path.join(settings.BASE_DIR, "cookies/facebook_cookie.pkl")
        self.lang_map = langauag_map = {
            "zh_hant": "中文(台灣)",
            "en": "English (US)",
            "zh_hans": "中文(简体)",
            "vi": "Tiếng Việt"
        }
        self.lang = self.lang_map.get(lang)
        
    def create_driver(self):
        self.driver = webdriver.Chrome(options=self.chrome_options, service= self.chromeService)
        return self.driver
    
    def create_action_object(self):
        if self.driver:
            self.actions = ActionChains(self.driver)
        return self.actions
    
    def create_wait_object(self):
        if self.driver:
            self.wait = WebDriverWait(self.driver, 5)
        return self.wait
    
    def get_driver(self):
        return self.driver
    
    def find_cookies(self):
        if os.path.exists(self.cookies_path):
            cookies = pickle.load(open(self.cookies_path, "rb"))
            return cookies
        else:
            return None
    
    def login_with_cookies(self, cookies):
        print("login with cookies")
        for c in cookies:
            self.driver.add_cookie(c)
        self.is_login = True
        self.driver.refresh()
#         self.validate_user()
        
    def login(self, validate=False):
        print("login")
        count = 0
        while not self.is_login and count <= 4:
            if not validate:
                self.driver.get(self.login_url)
            email_input = self.driver.find_element(By.ID, "m_login_email")
            pass_input = self.driver.find_element(By.ID, "m_login_password")
            login_button = self.driver.find_element(By.NAME, "login")
            self.actions.send_keys_to_element(email_input, self.email)
            self.actions.send_keys_to_element(pass_input, self.password)
            self.actions.click(login_button)
            self.actions.perform()
            count += 1
            self.is_login = True
        time.sleep(2)
        self.validate_user()
        pickle.dump(self.driver.get_cookies(), open(self.cookies_path, "wb"))
    
    def validate_user(self):
        print("validate_user")
        # check if page pops up log in with one tap
        try:
            self.driver.find_element(By.CSS_SELECTOR, "input[value='regular_login']")
            ok_button = self.driver.find_element(By.CSS_SELECTOR, "button[value='OK']")
            self.actions.click(ok_button).perform()
            self.save_login = True
        except:
            pass
            
        if not self.save_login:
            # check new page identity double check
            try:
                my_profile = self.driver.find_element(By.CSS_SELECTOR, f"div[role='button']")
                self.actions.click(my_profile).perform()

                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                self.save_login = True
            except:
                pass
            
        if not self.save_login:
            # check header identity double check
            try:
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                pass_input = self.wait.until(
                    EC.presence_of_element_located((By.NAME, "pass"))
                )
                self.actions.send_keys_to_element(pass_input, self.password)
                login_button = self.driver.find_element(By.CSS_SELECTOR , "button[type='submit']")
                self.actions.click(login_button).perform()
                self.save_login = True
            except:
                pass
    def switch_language(self):
        self.driver.get("https://m.facebook.com/language/")
        current_language = self.driver.find_element(By.CLASS_NAME, "_5551")
    def get_post_shared_list_url(self):
        found = False
#         while self.loop_count <= 10:
        while not found:
            print(self.loop_count)
            self.driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            article = self.wait.until(
                    EC.visibility_of_element_located((By.TAG_NAME, "article"))
                )
            if article:
                print(8888)
                articles = self.driver.find_elements(By.XPATH, "//article")
               
                for article in articles:
                    if self.post_reference_id:
                        break
                    video_data = json.loads(article.get_attribute("data-ft"))
                    if video_data.get('video_id', None) != self.target_post_id:
                        continue
                    print(video_data)
                    post_data = json.loads(article.get_attribute("data-store"))
                    self.post_reference_id = str(post_data.get("feedback_target",""))
                    found = True
#                     for key,value in post_data.items():
#                         if key == "share_id":
# #                             print(key)
# #                             print(value)
#                             if ":" not in str(value):
#                                 post_id = str(value)
#                             else:
#                                 post_id = value.split(":")[1]
#                             if post_id != self.target_post_id:
#                                 break

#                         if key == "feedback_target":
# #                             print(key)
# #                             print(value)
#                             self.post_reference_id = str(value)
#                             break

            self.loop_count += 1
        print(self.post_shared_list_url + self.post_reference_id)
        return self.post_shared_list_url + self.post_reference_id
        
    def expand_all(self):
        try:
            while True:
                see_more_button = self.driver.find_element(By.ID, "m_more_item")
                if see_more_button:
                    self.actions.click(see_more_button).perform()
        except NoSuchElementException:
            print("Element is not present")
            
    def collect_names(self):
        names = set([el.text for el in self.driver.find_elements(By.TAG_NAME, "strong")])
        return names
    
    def start(self):
        start = arrow.now()
        try:
            self.create_driver()
            self.create_wait_object()
            self.create_action_object()
            
            cookies = self.find_cookies()
            if cookies:
                self.driver.get(self.page_url)
                self.login_with_cookies(cookies)
            else:    
                self.login()
                self.driver.get(self.page_url)
                
            post_shared_list_url = self.get_post_shared_list_url()
            self.driver.get(post_shared_list_url)
            
            if not self.save_login:
                self.validate_user()
            self.expand_all()
            name_list = self.collect_names()
            end = arrow.now()
            print(f"Spent: {end - start}")
#             self.driver.quit()
            return name_list
        except Exception as e:
            print(traceback.format_exc())
#             if self.driver:
#                 self.driver.quit()
            return None

In [219]:
page_username = "kuai.mai.2021"
target_post_id = "517999680166258"
language = "zh_hant"
langauag_map = {
    "zh_hant": "中文(台灣)",
    "en": "English (US)",
    "zh_hans": "中文(简体)",
    "vi": "Tiếng Việt"
}
fb_crawler = FacebookSharedListCrawler(page_username, target_post_id, language)

In [220]:
data = fb_crawler.start()
print(data)


login with cookies
0
8888
{'qid': '-5488559183431672677', 'mf_story_key': '517999680166258', 'top_level_post_id': '517999680166258', 'tl_objid': '517999680166258', 'content_owner_id_new': '100063671276299', 'throwback_story_fbid': '517999680166258', 'page_id': '109130787659110', 'story_location': 4, 'story_attachment_style': 'video_inline', 'ott': 'ottAX8K604IcoS4Ldfx', 'video_id': '517999680166258', 'sty': 38, 'ent_attachement_type': 'VideoAttachment', 'app_id': '6628568379', 'page_insights': {'100063671276299': {'page_id': '100063671276299', 'page_id_type': 'page', 'actor_id': '100063671276299', 'dm': {'isShare': 0, 'originalPostOwnerID': 0}, 'psn': 'EntVideoCreationStory', 'post_context': {'object_fbtype': 1, 'publish_time': 1664507961, 'story_name': 'EntVideoCreationStory', 'story_fbid': ['517999680166258']}, 'role': 1, 'sl': 4, 'targets': [{'actor_id': '100063671276299', 'page_id': '100063671276299', 'post_id': '517999680166258', 'role': 1, 'share_id': 0}]}}, 'profile_id': '100063